In [1]:
# Imports
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.callbacks import TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from keras.regularizers import l1,l2
from tensorflow import keras
import tensorflow as tf
from scipy import stats
import pandas as pd
import datetime, os
import numpy as np

Using TensorFlow backend.


In [ ]:
import lime
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function

In [10]:
dataFrame = pd.read_csv(r'data/p/NSP_23_attributes.csv')
dataFrame.head()


number_folds = 10
# Split the data in K-folds
kf = KFold(n_splits = number_folds, shuffle = True, random_state = 2)

for i in range(number_folds):
    result = next(kf.split(dataFrame), None)

    train =  dataFrame.iloc[result[0]]
    test  =  dataFrame.iloc[result[1]]
    
    y_test = test.pop('NSP') - 1
    X_test = np.array(test)
    
    
    y = train.pop('NSP') - 1
    X = np.array(train)
    num_features = X.shape[1]
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True)
    
    # binary encode
    y_train = keras.utils.to_categorical(y_train)
    y_val = keras.utils.to_categorical(y_val)
    
    # Make a new model, and train it!
    model = keras.Sequential([
    keras.layers.Dense(20, input_dim=num_features, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, 
                       activation='relu', 
                       kernel_regularizer=l2(0.01)),
    keras.layers.Dense(10, activation='relu', activity_regularizer=l1(0.01)),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(3, activation='softmax')
    ])


    opt = keras.optimizers.RMSprop(learning_rate=0.0005, rho=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    earlystop_callback = EarlyStopping( monitor='val_loss', 
                                        min_delta=0.0001,
                                        mode='min',
                                        patience=5
                                      )
    
    model.fit(x=X_train,
          y=y_train,
          epochs=1000,
          batch_size=50,
          validation_data=(X_val, y_val),
          callbacks=[earlystop_callback],
          verbose=0)
    
    y_pred = model.predict_classes(X_test)

    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, y_pred)
    # precision tp / (tp + fp)
    precision = np.around(precision_score(y_test, y_pred, average=None), decimals=3)
    # recall: tp / (tp + fn)
    recall = np.around(recall_score(y_test, y_pred, average=None), decimals=3)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = np.around(f1_score(y_test, y_pred, average=None), decimals=3)

    #model.summary()
    print("\n")
    print("-"*18)
    print('Accuracy: %f' % accuracy)
    print("-"*40)
    print("| Class:\t  N\t  S\t  P\t|")
    print("| Precision:\t", precision[0], "\t", precision[1], "\t", precision[2], "\t|")
    print("| Recall:\t", recall[0], "\t", recall[1], "\t", recall[2], "\t|")
    print("| F1:\t\t", f1[0], "\t", f1[1], "\t", f1[2], "\t|")
    print("-"*40)
    del model
    



------------------
Accuracy: 0.901408
----------------------------------------
| Class:	  N	  S	  P	|
| Precision:	 0.944 	 0.692 	 0.7 	|
| Recall:	 0.982 	 0.621 	 0.5 	|
| F1:		 0.963 	 0.655 	 0.583 	|
----------------------------------------


------------------
Accuracy: 0.906103
----------------------------------------
| Class:	  N	  S	  P	|
| Precision:	 0.949 	 0.696 	 0.733 	|
| Recall:	 0.976 	 0.552 	 0.786 	|
| F1:		 0.962 	 0.615 	 0.759 	|
----------------------------------------


------------------
Accuracy: 0.906103
----------------------------------------
| Class:	  N	  S	  P	|
| Precision:	 0.943 	 0.696 	 0.786 	|
| Recall:	 0.976 	 0.552 	 0.786 	|
| F1:		 0.96 	 0.615 	 0.786 	|
----------------------------------------


------------------
Accuracy: 0.920188
----------------------------------------
| Class:	  N	  S	  P	|
| Precision:	 0.954 	 0.733 	 0.9 	|
| Recall:	 0.971 	 0.759 	 0.643 	|
| F1:		 0.962 	 0.746 	 0.75 	|
-------------------------------------